In [ ]:
from NAPC import NAPC, conf
import mysql.connector
import pandas as pd

# MySQL connection parameters
mysql_config = {
    'user': 'user_1',
    'password': '1234',
    'host': 'localhost',
    'database': 'framework'
}

# update with your IBM Token
token = "put_your_ibm_token_here"

# Running to the noisy simulator

## Example of code of length N=16 (n=3) for logical $|+\rangle$

In [ ]:
# initialize the object using the provided token
q = NAPC(runs=conf.runs, token=token)

In [ ]:
# initialize the object using the provided token
q = NAPC(runs=conf.runs, token=token)

# command to run the update TriQ's reliability matrix for the noise-aware routing
q.update_hardware_configs()

# then specify which polar codes we want to run, for example polar codes for logical + with length N=16 (n=3)
file_path="./QEC/polar_code/n3/x"

qasm_files = q.get_qasm_files_from_path(file_path)

# then decide which compilation to be run Qiskit-3 or S-TriQ (triq_lcd_sabre)
compilations=["qiskit_3", "triq_lcd_sabre"]

# this is the setup to enable mid measurement for TriQ
triq_measurement_type="polar_meas"

# choosing the noise level
noise_levels = [0.1, 1.0]

# setup the number of shots
shots = 1000

# then we setup the backend and run the simulator
q.set_backend(program_type="sampler", shots=shots)
q.run_simulator("sampler", qasm_files, compilations, noise_levels, shots)

In [ ]:
# get the result and calculate the metric
q.get_qiskit_result("simulator")

**After this, you can check the result in the database**

# Running to the real backend

## Example of code of length N=32 (n=4) for logical $|0\rangle$

In [ ]:
# initialize the object using the provided token
q = NAPC(runs=conf.runs, token=token)

# command to run the update TriQ's reliability matrix for the noise-aware routing
q.update_hardware_configs()

# then specify which polar codes we want to run, for example polar codes for logical + with length N=16 (n=3)
file_path="./QEC/polar_code/n4/z"

qasm_files = q.get_qasm_files_from_path(file_path)

# then decide which compilation to be run Qiskit-3 or S-TriQ (triq_lcd_sabre)
compilations=["qiskit_3", "triq_lcd_sabre"]

# this is the setup to enable mid measurement for TriQ
triq_measurement_type="polar_meas"

# setup the number of shots
shots = 1000

# then we setup the backend and run the simulator
q.set_backend(program_type="sampler", shots=shots)



**The only different with sending to the simulator is this part**

In [ ]:
q.send_to_real_backend("sampler", qasm_files, compilations, shots=shots)

For other code of lengths simulation, see the file "**run_simulation.py**" and "**run_to_real_backend.py**"

# Accessing the result in the database

In [ ]:
conn = mysql.connector.connect(**mysql_config)
cursor = conn.cursor()

sql = """
SELECT header_id, circuit_name, compilation_name, noise_level, 
polar_count_accept, polar_count_logerror, polar_count_undecided, success_rate_polar, total_two_qubit_gate,
initial_mapping
FROM result WHERE header_id = 2;
""".format(1, "ibm_sherbrooke")

 # insert to circuit
cursor.execute(sql)

results = cursor.fetchall()

cursor.close()
conn.close()

In [ ]:
header_id = []
circuit_name = []
compilation_name = []
noise_level = []
count_accept = []
count_logerror = []
count_undecided = []
success_polar = []
total_gate_cx = []
initial_mappings = []


for res in results:
    _header_id, _circuit_name, _compilation_name , _noise_level, _count_accept, _count_logerror, \
    _count_undecided, _success_polar, _total_gate_cx, _initial_mappings = res

    header_id.append(_header_id)
    circuit_name.append(_circuit_name)
    compilation_name.append(_compilation_name)
    noise_level.append(float(_noise_level))
    count_accept.append(_count_accept)
    count_logerror.append(_count_logerror)
    count_undecided.append(_count_undecided)
    success_polar.append(1-float(_success_polar))
    total_gate_cx.append(float(_total_gate_cx))
    initial_mappings.append(_initial_mappings)

df = pd.DataFrame({
    'header_id': header_id,
    'circuit_name': circuit_name,
    'compilation_name': compilation_name,
    'noise_level': noise_level,
    'count_accept': count_accept,
    'count_logerror': count_logerror, 
    'count_undecided': count_undecided, 
    'success_polar': success_polar,
    'total_gate_cx': total_gate_cx,
    'noise_level': noise_level,
    'initial_mappings': initial_mappings
    })


In [ ]:
df.head()